In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import dask.dataframe as dd
from google.colab import drive
import gc
import plotly.graph_objects as go

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
urltt1 = '.../table_1.csv'
urltt2 = '.../table_2.csv'

In [4]:
df1 = pd.read_csv(urltt1)
df2 = pd.read_csv(urltt2)

In [5]:
gc.collect()

60

In [6]:
# Convert the pandas dataframes to dask dataframes
ddf1 = dd.from_pandas(df1, npartitions=10)
ddf2 = dd.from_pandas(df2, npartitions=10)

ddf1['date'] = dd.to_datetime(ddf1['date'])
ddf2['date'] = dd.to_datetime(ddf2['date'])

# Merge
merged_ddf = ddf1.merge(ddf2, on=['user_id', 'date'], how='left')

merged_ddf.to_csv('data.csv', mode='a', index=False)

['/content/data.csv']

In [7]:
# Save csv to output data folder
!cp data.csv /content/drive/MyDrive/nord_data

In [8]:
murl = '.../data.csv'
mdf = pd.read_csv(murl)

In [21]:
mdf

,date,season,season_day,global_user_id,registration_date,cohort_day,registration_country_name,registration_platform,last_session_country,last_session_platform,...,tokens_spent_on_auction_player,in_squad_scout_count,tokens_spent_on_scout,in_squad_daily_assistant_player_count,tokens_spent_on_daily_assistant_player,in_squad_recommended_player_count,tokens_spent_on_recommended_player,time_utc,reason,net_revenue
0,2023-01-01,166,1,4297972055,2020-11-09,783,Mauritius,Android,Mauritius,Android,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,2023-01-01,166,1,4299300951,2022-11-10,52,Ghana,Android,Ghana,Android,...,0,1,49,0,0,0,0,NaN,NaN,NaN
2,2023-01-01,166,1,4299468619,2022-12-31,1,Kenya,Android,Kenya,Android,...,20,0,0,0,0,1,7,NaN,NaN,NaN
3,2023-01-01,166,1,4299472223,2023-01-01,0,Cape Verde,Android,Cape Verde,Android,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,2023-01-01,166,1,4299473251,2023-01-01,0,"Tanzania, United Republic of",Android,"Tanzania, United Republic of",Android,...,0,0,0,0,0,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92443,2023-03-25,168,28,957782002835,2023-01-16,68,Ghana,iOS,Ghana,iOS,...,0,0,0,0,0,0,0,2023-03-25 19:42:31.000,Training Bonus Multipliers,0.000958
92444,2023-03-25,168,28,957782128498,2023-02-26,27,Cote D'Ivoire,iOS,Cote D'Ivoire,iOS,...,0,0,0,0,0,0,0,NaN,NaN,NaN
92445,2023-03-25,168,28,957782205080,2023-03-23,2,Angola,iOS,Angola,iOS,...,6,0,0,1,12,0,0,2023-03-25 01:16:58.000,Shop Money,0.002126
92446,2023-03-25,168,28,957782208822,2023-03-24,1,Mali,Android,Mali,Android,...,0,0,0,0,0,1,39,NaN,NaN,NaN


In [22]:
nan_counts = mdf.isna().sum()
print('Number of NaN values in each column:')
print(nan_counts)

Number of NaN values in each column:
date                                          0
season                                        0
season_day                                    0
global_user_id                                0
registration_date                             0
cohort_day                                    0
registration_country_name                     0
registration_platform                         0
last_session_country                         17
last_session_platform                        17
session_count_daily                           0
playtime_daily                                0
training_count                                0
iap_transaction_count_daily                   0
net_revenue_usd_iap_daily                     0
is_payer_lifetime                             0
first_transaction_date_iap                75083
tokens_earned                                 0
rest_boosters_earned                          0
morale_boosters_earned                        0
tre

In [23]:
# Separate users who watched ads
ad_users = mdf[~mdf['reason'].isna()]
non_ad_users = mdf[mdf['reason'].isna()]

In [24]:
# Adding special ads indicator
ad_users.loc[:, 'is_special_ad'] = ad_users['reason'].str.contains('-', na=False)
special_ad_count = ad_users['is_special_ad'].value_counts()[True]
regular_count = ad_users['is_special_ad'].value_counts()[False]

print('Number of special_ad users:', special_ad_count)
print('Number of regular ad users:', regular_count)

Number of special_ad users: 6753
Number of regular ad users: 76507


<ipython-input-24-8f397e448c24>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [25]:
ad_users

,date,season,season_day,global_user_id,registration_date,cohort_day,registration_country_name,registration_platform,last_session_country,last_session_platform,...,in_squad_scout_count,tokens_spent_on_scout,in_squad_daily_assistant_player_count,tokens_spent_on_daily_assistant_player,in_squad_recommended_player_count,tokens_spent_on_recommended_player,time_utc,reason,net_revenue,is_special_ad
5,2023-01-01,166,1,55838699769,2022-08-11,143,Libya,FB or Site,Libya,Android,...,0,0,0,0,1,19,2023-01-01 06:48:18.000,Shop Morales,0.000208,False
6,2023-01-01,166,1,55838699769,2022-08-11,143,Libya,FB or Site,Libya,Android,...,0,0,0,0,1,19,2023-01-01 06:49:18.000,Shop Morales,0.000197,False
7,2023-01-01,166,1,55838699769,2022-08-11,143,Libya,FB or Site,Libya,Android,...,0,0,0,0,1,19,2023-01-01 06:50:01.000,Shop Morales,0.000197,False
8,2023-01-01,166,1,55838699769,2022-08-11,143,Libya,FB or Site,Libya,Android,...,0,0,0,0,1,19,2023-01-01 06:50:48.000,Shop Morales,0.000159,False
9,2023-01-01,166,1,55838699769,2022-08-11,143,Libya,FB or Site,Libya,Android,...,0,0,0,0,1,19,2023-01-01 06:51:30.000,Shop Morales,0.000150,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92440,2023-03-25,168,28,957782002835,2023-01-16,68,Ghana,iOS,Ghana,iOS,...,0,0,0,0,0,0,2023-03-25 19:27:22.000,Physio Center Rest Multiple Players,0.001771,False
92441,2023-03-25,168,28,957782002835,2023-01-16,68,Ghana,iOS,Ghana,iOS,...,0,0,0,0,0,0,2023-03-25 19:28:07.000,Physio Center Rest Multiple Players,0.001021,False
92442,2023-03-25,168,28,957782002835,2023-01-16,68,Ghana,iOS,Ghana,iOS,...,0,0,0,0,0,0,2023-03-25 19:30:30.000,Training Bonus Multipliers,0.000958,False
92443,2023-03-25,168,28,957782002835,2023-01-16,68,Ghana,iOS,Ghana,iOS,...,0,0,0,0,0,0,2023-03-25 19:42:31.000,Training Bonus Multipliers,0.000958,False


In [26]:
nan_counts = ad_users.isna().sum()
print(nan_counts)

date                                          0
season                                        0
season_day                                    0
global_user_id                                0
registration_date                             0
cohort_day                                    0
registration_country_name                     0
registration_platform                         0
last_session_country                          0
last_session_platform                         0
session_count_daily                           0
playtime_daily                                0
training_count                                0
iap_transaction_count_daily                   0
net_revenue_usd_iap_daily                     0
is_payer_lifetime                             0
first_transaction_date_iap                66781
tokens_earned                                 0
rest_boosters_earned                          0
morale_boosters_earned                        0
treatment_boosters_earned               

In [30]:
# Average daily revenue from in-game purchaces, daily playtime, daily session count and transaction count - special ad vs regular ad watchers
groups = ad_users.groupby('is_special_ad')

average_revenue = groups['ads_net_revenue_purchaces_daily'].mean()
average_playtime = groups['playtime_daily'].mean()
average_session_count = groups['session_count_daily'].mean()
average_transaction_count = groups['transaction_count_daily'].mean()

print('Average revenue by special_ad watchers:', average_revenue[True])
print('Average revenue by regular-ad watchers:', average_revenue[False])
print('Average daily playtime by special_ad watchers:', average_playtime[True])
print('Average daily playtime by regular-ad watchers:', average_playtime[False])
print('Average daily session count by special_ad watchers:', average_session_count[True])
print('Average daily session count by regular-ad watchers:', average_session_count[False])
print('Average daily transaction count by special_ad watchers:', average_transaction_count[True])
print('Average daily transaction count by regular-ad watchers:', average_transaction_count[False])

Average revenue by special_ad watchers: 0.09577250166108613
Average revenue by regular-ad watchers: 0.08856478860761281
Average daily playtime by special_ad watchers: 5299164.458462905
Average daily playtime by regular-ad watchers: 5403467.616388043
Average daily session count by special_ad watchers: 9.196357174589071
Average daily session count by regular-ad watchers: 9.079482923131216
Average daily transaction count by special_ad watchers: 0.01776988005330964
Average daily transaction count by regular-ad watchers: 0.020259584090344673


In [31]:
groups = ad_users.groupby('is_special_ad')
average_iap_revenue = groups['ads_net_revenue_purchaces_daily'].mean()
average_ad_revenue = groups['ads_net_revenue'].mean()
average_playtime = groups['playtime_daily'].mean()
average_session_count = groups['session_count_daily'].mean()
average_transaction_count = groups['transaction_count_daily'].mean()

metrics = ['Daily Net Revenue from in Game Purchases', 'Net Revenue from Ads', 'Daily Playtime in miliseconds', 'Daily Session Count', 'Daily Transaction Count']

mean_values_special_ad = [average_iap_revenue[True], average_ad_revenue[True], average_playtime[True], average_session_count[True], average_transaction_count[True]]
mean_values_regular = [average_iap_revenue[False], average_ad_revenue[False], average_playtime[False], average_session_count[False], average_transaction_count[False]]

for i in range(len(metrics)):
    fig = go.Figure(data=[
        go.Bar(name='special_ad watchers', x=['special_ad'], y=[mean_values_special_ad[i]]),
        go.Bar(name='Regular-ad watchers', x=['Regular'], y=[mean_values_regular[i]])
    ])
    fig.update_layout(barmode='group', title_text=f'Average {metrics[i]} by special_ad watchers and Regular-ad watchers')

    fig.show(config={'scrollZoom': True})


In [32]:
# Cross-tabulation
cross_tab = pd.crosstab(ad_users['is_special_ad'], ad_users['is_payer'])
print(cross_tab)

is_payer_lifetime  False  True 
is_special_ad                  
False              61535  14972
True                5246   1507


In [33]:
cross_tab = pd.crosstab(ad_users['is_special_ad'], ad_users['is_payer'])

fig = go.Figure()

fig.add_trace(go.Bar(
    x=['AD', 'special_ad'],
    y=cross_tab[True],
    name='Made purchase in game',
    marker_color='blue'
))

fig.add_trace(go.Bar(
    x=['AD', 'special_ad'],
    y=cross_tab[False],
    name='Did not make purchace in game',
    marker_color='red'
))

fig.update_layout(barmode='stack', title_text='Purchase Behavior Comparison: special_ad vs. AD Users')

fig.update_layout(
    legend=dict(
        title='is_payer',
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1
    )
)


fig.show()


In [34]:
ad_users_filtered = ad_users[ad_users['is_payer'] == False]

special_ad_watchers = ad_users_filtered[ad_users_filtered['is_special_ad'] == True]

regular_watchers = ad_users_filtered[ad_users_filtered['is_special_ad'] == False]

average_special_ad_revenue = special_ad_watchers['ads_net_revenue'].mean()
average_regular_revenue = regular_watchers['ads_net_revenue'].mean()

fig = go.Figure(data=[
   go.Bar(name='special_ad', x=['special_ad'], y=[average_special_ad_revenue]),
   go.Bar(name='Regular-ad', x=['AD'], y=[average_regular_revenue])
])
fig.update_layout(barmode='group', title_text='Average Net Revenue from ads (special_ad non-payers and regular-ad non-payers')

fig.show(config={'scrollZoom': True})
